In [ ]:
using JuMP, SDDP, HiGHS, Gurobi
using DataStructures
using Distributions, Statistics
using Test
using HDF5

# Performance of SDDP

In [ ]:
# In Julia this struct is immutable
struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nSpotLanes      ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    SpotIdx         ::Vector{Vector{Int64}}
    # from_           ::Vector{Vector{Int64}}
    from_           ::Dict{Symbol, Vector{Vector{Int64}}}
    # to_             ::Vector{Vector{Int64}}
    to_             ::Dict{Symbol, Vector{Vector{Int64}}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    spot_coef       ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
function HyperParams(; 
                     tau             ::Int64 = 5,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 4],
                                                                [2, 3],
                                                                [1, 3, 4],
                                                                [2, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [200, 200],
                     exit_stock_0    ::Vector{Int64} = [0, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [1000, 1000],
                     exit_capacity   ::Vector{Int64} = [1000, 1000],
                     flow_support    ::Vector{Int64} = collect(100:10:500),
                     entry_store_coef::Vector{Float64} = [20.0, 20.0],
                     exit_store_coef ::Vector{Float64} = [10.0, 10.0],
                     exit_short_coef ::Vector{Float64} = [30.0, 30.0],
                     transport_coef  ::Vector{Float64} = [ 4.8, 3.3, 6, 6.9, 7.3, 5.9, 5.8, 5.4, 3.3, 6.4, 6.9, 5.2, 6.3 ], 
                     spot_coef       ::Vector{Float64} = [2.80, 2.85, 3.30, 3.84, 3.39, # Lane 1 - Carrier 1
                                                          3.58, 3.37, 2.64, 2.81, 4.31, # Lane 2 - Carrier 1
                                                          4.47, 4.07, 3.76, 4.20, 2.95, # Lane 3 - Carrier 1
                                                          3.79, 4.02, 2.56, 3.55, 4.69, # Lane 4 - Carrier 1
                                                          4.63, 2.84, 4.41, 4.91, 4.79, # Lane 1 - Carrier 2
                                                          3.44, 4.83, 3.70, 2.94, 2.83, # Lane 2 - Carrier 2
                                                          4.60, 4.53, 3.39, 2.89, 3.87, # Lane 3 - Carrier 2
                                                          2.82, 4.50, 4.97, 4.85, 4.48, # Lane 4 - Carrier 2
                                                          2.64, 4.55, 3.53, 3.26, 3.76, # Lane 1 - Carrier 3
                                                          4.53, 4.40, 3.25, 3.97, 4.71, # Lane 2 - Carrier 3
                                                          4.61, 3.15, 3.24, 4.15, 2.52, # Lane 3 - Carrier 3
                                                          4.40, 3.89, 3.79, 3.21, 3.78],# Lane 4 - Carrier 3
                     carrier_capacity::Vector{Int64} = [160, 100, 140, 120, 100,  # Carrier 1
                                                        140, 100, 160, 180, 100,  # Carrier 2
                                                        100, 100, 100, 100, 100,  # Carrier 3
                                                         30,  30,  30,  30,  30,  # Carrier 1
                                                         30,  30,  30,  30,  30,  # Carrier 2
                                                         30,  30,  30,  30,  30], # Carrier 3
                     N::Int64 = 100)

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)
    spotLdx = repeat(1:(nOrigins * nDestinations), nCarriers) # Spot

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    strategicIdx = [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ]
    spotIdx      = [ collect(((i - 1)*(nOrigins * nDestinations) + 1):i * (nOrigins * nDestinations)) for i in 1:nCarriers ]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nOrigins * nDestinations * nCarriers, # nSpotLanes (exhaustive)
        nCarriers,
        Bids,
        Winners,
        [ strategicIdx; [nLc[end] .+ idx for idx in spotIdx] ],   # CarrierIdx
        spotIdx, # Spotdx
        # [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],    # from_
        Dict(
            :SG => [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ], 
            :SP => [ findall(lane -> lane in from_[i], spotLdx) .+ nLc[end] for i in 1:nOrigins ]),
        # [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ], # to_
        Dict(
            :SG => [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ], 
            :SP => [ findall(lane -> lane in to_[j], spotLdx) .+ nLc[end] for j in 1:nDestinations ]),
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        # vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))), # Exhaustive 
        unique([ntuple(_ -> rand(flow_support), nOrigins + nDestinations) for _ in 1:N]), # N draws from the exhaustive sample
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        spot_coef,
        carrier_capacity)
end

In [ ]:
function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:(config.nLanes + config.nSpotLanes)]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity (Contract and Spot)
    @constraint(sp, [k = 1:(2 * config.nCarriers)], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # Entry volume
    @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[:SG][i]]) + sum(move[config.from_[:SP][i]]) <= entry[i].in + inflow[i])
    # Storage limit
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[:SG][j]]) + sum(move[config.to_[:SP][j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[:SG][i]]) - sum(move[config.from_[:SP][i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[:SG][j]]) + sum(move[config.to_[:SP][j]]) - outflow[j])
    # Uncertain variables
    Ξ = config.flow_support[rand(1:length(config.flow_support), 10)] # Training with uniform sampling (distribution agnostic)
    # Ξ = [ rand(Poisson(1500), config.nOrigins + config.nDestinations) for _ in 1:10 ] # No need for that!
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    kdx = vec(((1:config.nCarriers)' .- 1) * config.nOrigins * config.nDestinations * config.tau .+ (config.SpotIdx[1] .- 1) * config.tau .+ stage)
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        # sum(sum(config.transport_coef[config.CarrierIdx[k]] .* move[config.CarrierIdx[k]]) for k in 1:config.nCarriers) + 
        sum(config.transport_coef[1:config.nLanes] .* move[1:config.nLanes]) + 
        # sum(sum(config.spot_coef[(k - 1) * config.nOrigins * config.nDestinations * config.tau .+ (config.SpotIdx[1] .- 1) * config.tau .+ stage] .* move[config.CarrierIdx[config.nCarriers + k]]) for k in 1:config.nCarriers)
        sum(config.spot_coef[kdx] .* move[(config.nLanes + 1):(config.nLanes + config.nSpotLanes)])
    )
end

## 4 time periods with 4x4 ODs and 3 carriers

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau             = 4,
    nOrigins        = 4,
    nDestinations   = 4,
    nCarriers       = 3,
    Bids            = [[ 1,  2,  3,  4,  5,  6,  7,  8],
                       [ 5,  6,  7,  8,  9, 10, 11, 12],
                       [10, 11, 12, 13, 14, 15, 16],
                       [ 1,  2,  3,  4,  8,  9, 10, 11],
                       [ 5,  6,  7,  8, 13, 14, 15, 16]],
    entry_stock_0   = [500, 500, 500, 500],
    exit_stock_0    = [0, 0, 0, 0],
    exit_short_0    = [0, 0, 0, 0],
    entry_capacity  = [1000, 1000, 1000, 1000],
    exit_capacity   = [1000, 1000, 1000, 1000],
    flow_support    = collect(0:100:500),
    entry_store_coef= [2.0, 2.0, 2.0, 2.0],
    exit_store_coef = [1.0, 1.0, 1.0, 1.0],
    exit_short_coef = [3.0, 3.0, 3.0, 3.0],
    transport_coef  = [ 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3,   # Carrier 1 / Lane 1 - 16
                        1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7,   # Carrier 2 / Lane 1 - 16
                        2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1 ], # Carrier 3 / Lane 1 - 16
    carrier_capacity= [160, 100, 140, 120,  # Carrier 1 / Time Stage 1 - 4
                       140, 100, 160, 180,  # Carrier 2 / Time Stage 1 - 4
                       300, 300, 300, 300 ] # Carrier 3 / Time Stage 1 - 4
);

## 12 time periods with 4x4 ODs and 3 carriers

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau             = 12,
    nOrigins        =  4,
    nDestinations   =  4,
    nCarriers       =  3,
    Bids            = [[ 1,  2,  3,  4,  5,  6,  7,  8],
                       [ 5,  6,  7,  8,  9, 10, 11, 12],
                       [10, 11, 12, 13, 14, 15, 16],
                       [ 1,  2,  3,  4,  8,  9, 10, 11],
                       [ 5,  6,  7,  8, 13, 14, 15, 16]],
    entry_stock_0   = [200, 100, 50,   0],
    exit_stock_0    = [  0, 100,  0, 100],
    exit_short_0    = [  0,   0,  0,   0],
    entry_capacity  = [1000, 1000, 1000, 1000],
    exit_capacity   = [1000, 1000, 1000, 1000],
    flow_support    = collect(0:500), # [  0, 100, 300, 500],
    entry_store_coef= [2.0, 2.0, 2.0, 2.0],
    exit_store_coef = [1.0, 1.0, 1.0, 1.0],
    exit_short_coef = [3.0, 3.0, 3.0, 3.0],
    transport_coef  = [2.89, 3.94, 3.77, 3.89, 3.29, 2.05, 5.79, 5.97, 5.69, 4.39, 
                       5.54, 3.74, 4.67, 2.03, 2.22, 5.44, 4.94, 4.75, 2.64, 3.25, 
                       5.64, 2.00, 5.51, 2.84, 3.15, 3.77, 4.95, 2.32, 5.35, 5.20, 
                       5.15, 5.68, 4.75, 3.68, 3.23, 3.03, 2.13, 4.55, 4.85],
    carrier_capacity= [150, 150, 100, 150, 150, 100, 100,  50, 150, 150,  50,  50, 
                       100, 150, 100,  50, 150, 100, 100,  50, 150, 150, 100, 100, 
                       100, 100, 150,  50, 100, 150,  50, 150,  50, 150, 100,  50 ],
    N = 10000000
);

## Train

In [ ]:
config = HyperParams();

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
@time SDDP.train(model;
    iteration_limit = 1000,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

In [ ]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    1000,
    # A list of names to record the values of.
    [:move, :inflow, :outflow, :entry, :exitp, :exitm],
);

In [ ]:
# Simulated objectives
obj = map(sim -> sum(node[:stage_objective] for node in sim), simulations);

# Scenarios
noise = vcat([ vcat([collect(ξ)' for ξ in map(node -> node[:noise_term], sim)]...) for sim in simulations ]...);

In [ ]:
h5write(joinpath(pwd(), "output/5x2x2x3_sims.h5"), "5x2x2x3_trials10_obj_sims10e3", obj)
h5write(joinpath(pwd(), "output/5x2x2x3_sims.h5"), "5x2x2x3_trials10_ksi_sims10e3", noise)

In [ ]:
KsiN = reduce(vcat, [collect(v)' for v in config.flow_support]);

In [ ]:
vsbs = collect(setdiff(Set(eachrow(KsiN)), Set(eachrow(unique(noise, dims=1)))));
out_of_bag = hcat([collect(sbarr) for sbarr in vsbs]...)';

In [ ]:
idx = sample(1:size(out_of_bag, 1), 1000 * config.tau, replace=true);

In [ ]:
out_of_bag_N = [ [(t, out_of_bag[idx[(i - 1) * config.tau + t],:]) for t in 1:config.tau] for i in 1:1000 ];

In [ ]:
simulxtions = SDDP.simulate(
    model, 
    1000,
    [:move, :inflow, :outflow, :entry, :exitp, :exitm]; 
    sampling_scheme = SDDP.Historical(out_of_bag_N), 
    parallel_scheme = SDDP.Threaded());

In [ ]:
# Simulated objectives
objx = map(sim -> sum(node[:stage_objective] for node in sim), simulxtions);

# Scenarios
noisex = vcat([ vcat([collect(ξ)' for ξ in map(node -> node[:noise_term], sim)]...) for sim in simulxtions ]...);

In [ ]:
h5write(joinpath(pwd(), "output/5x2x2x3_sims.h5"), "5x2x2x3_trials10_obj_oob10e3", objx)
h5write(joinpath(pwd(), "output/5x2x2x3_sims.h5"), "5x2x2x3_trials10_ksi_oob10e3", noisex)

# Scaling to realistic instances

In [ ]:
using JSON3

In [ ]:
ic = JSON3.read(joinpath(pwd(), "instances/instance_12x6x6x20_001.json"));
# Hyperparameter configuration
config = HyperParams(
    tau             = ic[:tau][1],
    nOrigins        = ic[:nOrigins][1],
    nDestinations   = ic[:nDestinations][1],
    nCarriers       = ic[:nCarriers][1],
    Bids            = map(x -> Vector{Int64}(x), ic[:Bids]),
    Winners         = OrderedDict(k => Vector{Int64}(ic[:winners][k]) for k in 1:length(ic[:winners])),
    entry_stock_0   = Vector{Int64}(ic[:entry_stock_0]),
    exit_stock_0    = Vector{Int64}(ic[:exit_stock_0]),
    exit_short_0    = Vector{Int64}(ic[:exit_short_0]),
    entry_capacity  = Vector{Int64}(ic[:entry_capacity]), # [10000, 10000, 10000, 10000, 10000, 10000], 
    exit_capacity   = Vector{Int64}(ic[:exit_capacity]),  # [10000, 10000, 10000, 10000, 10000, 10000], 
    flow_support    = Vector{Int64}(ic[:flow_support]),
    entry_store_coef= Vector{Float64}(ic[:entry_store_coef]),
    exit_store_coef = Vector{Float64}(ic[:exit_store_coef]),
    exit_short_coef = Vector{Float64}(ic[:exit_short_coef]),
    transport_coef  = Vector{Float64}(ic[:transport_coef]),
    spot_coef       = Vector{Float64}(ic[:spot_coef]),
    carrier_capacity= Vector{Int64}(ic[:carrier_capacity]),
    N = 10000000
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 1500,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

In [ ]:
trials = 1000;

In [ ]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    trials,
    # A list of names to record the values of.
    [:move, :inflow, :outflow, :entry, :exitp, :exitm],
);

In [ ]:
# Simulated objectives
obj = map(sim -> sum(node[:stage_objective] for node in sim), simulations);

# Scenarios
noise = vcat([ vcat([collect(ξ)' for ξ in map(node -> node[:noise_term], sim)]...) for sim in simulations ]...);

In [ ]:
h5write(joinpath(pwd(), "output/12x6x6x20_10x1000_sims.h5"), "12x6x6x20_Unif10x1500_obj_sims10e3", obj)
h5write(joinpath(pwd(), "output/12x6x6x20_10x1000_sims.h5"), "12x6x6x20_Unif10x1500_ksi_sims10e3", noise)

In [ ]:
KsiN = reduce(vcat, [collect(v)' for v in config.flow_support]);

vsbs = collect(setdiff(Set(eachrow(KsiN)), Set(eachrow(unique(noise, dims=1)))));
out_of_bag = hcat([collect(sbarr) for sbarr in vsbs]...)';

idx = sample(1:size(out_of_bag, 1), trials * config.tau, replace=false);
out_of_bag_N = [ [(t, out_of_bag[idx[(i - 1) * config.tau + t],:]) for t in 1:config.tau] for i in 1:trials ];

In [ ]:
# # Different out-of-sample distribution
# out_of_bag_N = [ [(t, rand(Poisson(1500), config.nOrigins + config.nDestinations)) for t in 1:config.tau] for i in 1:trials ];

In [ ]:
simulxtions = SDDP.simulate(
    model, 
    trials,
    [:move, :inflow, :outflow, :entry, :exitp, :exitm]; 
    sampling_scheme = SDDP.Historical(out_of_bag_N), 
    parallel_scheme = SDDP.Threaded());

In [ ]:
# Simulated objectives
objx = map(sim -> sum(node[:stage_objective] for node in sim), simulxtions);

# Scenarios
noisex = vcat([ vcat([collect(ξ)' for ξ in map(node -> node[:noise_term], sim)]...) for sim in simulxtions ]...);

In [ ]:
h5write(joinpath(pwd(), "output/12x6x6x20_10x1000_sims.h5"), "12x6x6x20_Unif10x1500_obj_oob10e3", objx)
h5write(joinpath(pwd(), "output/12x6x6x20_10x1000_sims.h5"), "12x6x6x20_Unif10x1500_ksi_oob10e3", noisex)

# Realistic uncertainty model

# Spot market uncertainty

# Manual instance

In [ ]:
# # Hyperparameter configuration
# config = HyperParams(
#     tau             = 12,
#     nOrigins        = 6,
#     nDestinations   = 6,
#     nCarriers       = 20,
#     Bids            = [[6, 23, 35, 21, 4, 1, 7, 17, 33, 25, 29, 32, 13, 30, 31, 14],
#                        [20, 28, 16, 27, 31, 25, 35, 2, 1, 14, 18, 9, 34, 10],
#                        [28, 5, 24, 21, 26, 19, 33, 20, 8, 2, 16, 29],
#                        [16, 20, 32, 35, 30, 19],
#                        [8, 21, 3, 23, 24, 28, 27, 30, 31, 1, 33, 9, 29],
#                        [35, 23, 18, 17, 31, 10, 2, 16, 9, 6, 11, 1], 
#                        [33, 31, 28, 4, 29, 30, 5, 16, 9, 20, 1, 36, 32], 
#                        [18, 24, 32, 31, 26, 25, 23, 19, 36, 20, 27, 29, 1, 34, 11], 
#                        [13, 20, 14, 29, 27, 8, 26, 19, 11, 12, 6, 24, 5, 34, 17],
#                        [9, 12, 6, 8, 29, 1, 36, 4, 33, 30, 17]
#                       ],
#     Winners         = OrderedDict(
#                          1 => [10, 6], 
#                          2 => [8], 
#                          3 => [3, 6], 
#                          4 => [3], 
#                          5 => [7], 
#                          6 => [8], 
#                          7 => [7], 
#                          8 => [9], 
#                          9 => [6], 
#                         10 => [8, 6], 
#                         11 => [2], 
#                         12 => [6, 1], 
#                         13 => [4, 3], 
#                         14 => [1, 3], 
#                         15 => [5], 
#                         16 => [1, 4], 
#                         17 => [3, 8], 
#                         18 => [5], 
#                         19 => [1, 4], 
#                         20 => [1] ),
#     entry_stock_0   = [200, 100,  50,   0,   0, 100],
#     exit_stock_0    = [100,   0,   0, 200, 100, 100],
#     exit_short_0    = [  0,   0,   0,   0,   0,   0],
#     entry_capacity  = [5000, 5000, 5000, 5000, 5000, 5000],
#     exit_capacity   = [5000, 5000, 5000, 5000, 5000, 5000],
#     flow_support    = collect(1000:100:3000),
#     entry_store_coef= [20.0, 20.0, 20.0, 20.0, 20.0, 20.0],
#     exit_store_coef = [10.0, 10.0, 10.0, 10.0, 10.0, 10.0],
#     exit_short_coef = [30.0, 30.0, 30.0, 30.0, 30.0, 30.0],
#     transport_coef  = [
#         6.21, 6.67, 5.08, 8.67, 5.09, 5.63, 7.65, 4.90, 5.70, 7.38, 6.78, 6.22, 5.69, 5.62, 7.25, 
#         6.31, 5.47, 5.21, 4.70, 4.34, 7.07, 5.94, 6.42, 5.87, 4.16, 4.64, 5.90, 6.20, 6.81, 7.47, 
#         5.85, 5.66, 5.05, 5.10, 6.37, 5.84, 6.89, 5.13, 4.79, 4.45, 6.31, 7.21, 6.75, 6.96, 5.99, 
#         5.61, 5.46, 4.99, 6.07, 4.95, 5.78, 7.35, 3.58, 7.44, 5.50, 5.90, 7.30, 7.22, 5.65, 6.83, 
#         6.59, 6.83, 7.48, 7.06, 6.27, 7.93, 6.19, 5.52, 5.17, 5.72, 5.51, 8.71, 6.02, 5.26, 7.42, 
#         5.38, 7.84, 6.06, 5.43, 5.30, 6.14, 6.94, 3.57, 6.47, 5.88, 4.52, 8.06, 6.03, 4.86, 5.00, 
#         6.45, 2.15, 6.22, 5.32, 5.84, 6.22, 5.56, 4.78, 5.94, 5.90, 5.78, 5.63, 6.49, 7.23, 4.94, 
#         6.24, 6.60, 4.64, 5.80, 6.78, 6.28, 5.19, 6.76, 7.21, 5.86, 5.98, 7.63, 5.39, 5.44, 4.61, 
#         5.22, 4.97, 5.32, 6.36, 6.02, 6.13, 5.58, 6.10, 5.85, 6.69, 4.86, 5.69, 6.92, 6.56, 7.03, 
#         4.71, 5.96, 6.04, 4.23, 7.58, 5.20, 6.90, 5.21, 7.00, 7.62, 6.28, 7.47, 6.00, 5.89, 7.86, 
#         4.91, 5.72, 6.91, 6.75, 5.49, 5.79, 6.33, 7.00, 6.26, 6.13, 7.24, 8.42, 8.72, 4.80, 6.20, 
#         6.49, 7.42, 5.84, 5.80, 5.90, 5.97, 4.87, 6.35, 4.98, 7.63, 5.61, 6.74, 6.41, 6.77, 7.06, 
#         4.70, 4.29, 5.74, 6.56, 6.21, 6.13, 6.01, 6.25, 7.97, 5.24, 4.66, 7.52, 5.19, 4.95, 6.19, 
#         6.49, 7.39, 6.05, 4.35, 7.27, 5.73, 5.58, 4.81, 4.01, 5.72, 5.03, 5.91, 6.02, 8.06, 7.10, 
#         5.57, 6.41, 5.44, 5.46, 6.48, 5.14, 6.26, 7.47, 4.90, 8.29, 6.17, 7.41, 5.74, 6.65, 6.75, 
#         6.30, 4.82, 5.66, 5.54, 5.47, 7.27, 4.56, 6.86, 4.24, 6.95, 6.73, 4.62, 6.34, 6.70, 4.48, 
#         7.89, 6.46, 4.94, 5.69, 4.87, 6.20, 4.90, 5.24, 8.35, 5.30, 6.03, 7.68, 5.21, 5.48, 6.13, 
#         4.24, 5.46, 6.08, 5.53, 4.41, 5.21, 5.98, 5.95, 4.76, 8.25, 6.01, 6.14, 6.94, 4.80, 6.74, 
#         6.79, 5.72, 7.94, 6.06, 6.32, 7.41, 8.44, 6.27, 4.90, 7.82, 5.30, 6.92, 4.92, 5.81, 5.26, 
#         4.59, 7.24, 5.63, 6.56, 5.43, 5.95, 6.15, 5.53, 5.06, 6.54, 4.97, 7.40, 5.77, 6.28, 6.78, 
#         4.60, 5.54, 4.83, 6.61, 5.32, 4.89, 2.63, 5.90, 6.57, 3.56, 6.98, 8.08, 6.26, 5.67, 6.50, 
#         5.22, 3.43, 7.05, 5.20, 5.22, 6.49, 6.40, 7.20, 5.56, 6.21, 5.85, 3.13, 4.25, 7.08, 7.50, 
#         6.60, 2.99, 4.88, 6.00, 6.31, 5.61, 4.77, 6.67, 6.76, 5.01, 5.06, 5.25, 4.40, 4.88, 6.50, 
#         5.97, 5.80, 5.31, 6.16, 5.99, 6.85, 5.78, 7.32, 6.84, 5.62, 7.18, 4.34, 5.44, 5.07, 5.74, 
#         6.31, 5.87, 5.28, 6.77, 5.47, 6.39, 5.83, 7.07, 4.30, 5.28], # map(x -> round(x, digits = 2), rand(Uniform(4.0, 8.0), 225)), # Carrier 3 / Lane 1 - 16
#     carrier_capacity= [
#         800, 700, 700, 500, 700, 500, 700, 800, 400, 400, 500, 700, 
#         400, 500, 500, 800, 500, 700, 500, 400, 400, 700, 800, 500, 
#         700, 800, 800, 500, 500, 800, 400, 800, 800, 700, 500, 500, 
#         600, 400, 500, 800, 800, 700, 700, 400, 400, 700, 700, 400, 
#         400, 400, 500, 700, 600, 600, 500, 700, 600, 600, 500, 700, 
#         800, 400, 500, 600, 700, 400, 800, 400, 600, 800, 400, 700, 
#         800, 500, 600, 700, 800, 400, 800, 800, 700, 400, 600, 400, 
#         700, 600, 700, 500, 400, 500, 600, 400, 600, 700, 700, 800, 
#         400, 600, 500, 800, 600, 500, 500, 400, 800, 700, 800, 500, 
#         400, 500, 700, 700, 500, 400, 400, 400, 700, 800, 500, 700, 
#         400, 800, 400, 500, 400, 500, 500, 700, 800, 600, 400, 700, 
#         800, 600, 800, 400, 800, 500, 400, 700, 400, 500, 600, 700, 
#         500, 800, 700, 800, 500, 700, 400, 400, 800, 400, 600, 600, 
#         800, 400, 500, 800, 600, 800, 500, 400, 500, 600, 800, 800, 
#         500, 800, 500, 700, 700, 600, 500, 400, 800, 800, 600, 500, 
#         600, 800, 800, 600, 800, 800, 400, 600, 500, 500, 500, 800, 
#         500, 700, 800, 400, 500, 600, 500, 800, 600, 500, 500, 800, 
#         600, 500, 700, 800, 700, 700, 800, 800, 500, 500, 400, 700, 
#         700, 700, 800, 800, 500, 600, 600, 500, 700, 700, 500, 800, 
#         700, 600, 700, 700, 700, 800, 600, 400, 600, 500, 700, 600
#     ], # rand(30:70, 20 * 12)
#     N = 10000000
# );